In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
import json
import glob
import os
from typing import List
import nbformat as nbf


In [2]:
df = pd.read_json('../raw_data/AI4Code/test_data/0010483c12ba9b.json')
df

,cell_type,source
54c7cab3,code,%reset -f \n\nif 1:\n # https://www.kaggle....
fe66203e,code,#config \n\ndiscourse_marker_to_label = {\n ...
7844d5f8,code,#data\n\ndf_text=[]\nfor id in valid_id:\n ...
5ce8863c,code,#net\n\nfrom bigbird_base_model import Net as ...
4a0777c4,code,#processing\n\ndef text_to_word(text):\n wo...
4703bb6d,code,## main submission function !!!!\n\n\ndef run_...
4a32c095,code,#check function\ndef run_check_dataset():\n\n ...
865ad516,code,# '''\n# cross validation results \n# WITHOUT ...
02a0be6d,code,#run_check_dataset()\nrun_submit()
7f270e34,markdown,This notebook illustrate how to speedup infere...


In [15]:
!pwd

/home/ilhanb11/code/ste94000/google_ai4_code/notebooks


In [19]:
output_path = '/home/ilhanb11/code/ste94000/google_ai4_code/notebooks/newntk.ipynb'

In [20]:
def dataframe_to_notebook(df: pd.DataFrame, output_path: str):
    # Create a new notebook object
    nb = nbf.v4.new_notebook()

    # Iterate over DataFrame rows and create cells
    cells = []
    for index, row in df.iterrows():
        cell_type = row['cell_type']
        source = row['source']

        if cell_type == 'markdown':
            cell = nbf.v4.new_markdown_cell(source)
        elif cell_type == 'code':
            cell = nbf.v4.new_code_cell(source)
        else:
            raise ValueError(f"Unsupported cell type: {cell_type}")

        cells.append(cell)

    # Assign cells to the notebook
    nb['cells'] = cells

    # Write the notebook to a file
    with open(output_path, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

In [21]:
dataframe_to_notebook(df, output_path=output_path)

In [18]:
df.shape

(10, 2)

In [42]:
notebook_path='../notebooks/test_notebook.ipynb'

In [62]:
def generate_cell_id():
    return uuid.uuid4().hex[:8]

def convert_notebook(notebook_path: str) -> pd.DataFrame:

    with open(notebook_path, 'r', encoding='utf-8') as file:
        notebook_json = json.load(file)

    cells = notebook_json['cells']
    data = []
    for cell in cells:
        cell_type = cell['cell_type']
        source = ''.join(cell['source'])  # Join list of strings into a single string
        cell_id = generate_cell_id()
        data.append({'cell_id': cell_id, 'cell_type': cell_type, 'source': source})

    return(pd.DataFrame(data).assign(id=os.path.basename(notebook_path).split(".")[0]).set_index('cell_id'))

def clean_code(cell: str) -> str:
    return str(cell).replace("\\n", "\n")


def sample_cells(cells: List[str], n: int) -> List[str]:
    cells = [clean_code(cell) for cell in cells]
    if n >= len(cells):
        return cells
    else:
        results = []
        step = len(cells) / n
        idx = 0
        while int(np.round(idx)) < len(cells):
            results.append(cells[int(np.round(idx))])
            idx += step
        if cells[-1] not in results:
            results[-1] = cells[-1]
        return results

def get_features(df: pd.DataFrame) -> dict:
    features = {}
    for i, sub_df in tqdm(df.groupby("id"), desc="Features"):
        features[i] = {}
        total_md = sub_df[sub_df.cell_type == "markdown"].shape[0]
        code_sub_df = sub_df[sub_df.cell_type == "code"]
        total_code = code_sub_df.shape[0]
        codes = sample_cells(code_sub_df.source.values, 20)
        features[i]["total_code"] = total_code
        features[i]["total_md"] = total_md
        features[i]["codes"] = codes
    return features


def tokenize(df: pd.DataFrame, fts: dict) -> dict:
    input_ids = np.zeros((len(df), TOTAL_MAX_LEN), dtype=np.int32)
    attention_mask = np.zeros((len(df), TOTAL_MAX_LEN), dtype=np.int32)
    features = np.zeros((len(df),), dtype=np.float32)

    for i, row in tqdm(
        df.reset_index(drop=True).iterrows(), desc="Tokens", total=len(df)
    ):
        row_fts = fts[row.id]

        inputs = TOKENIZER.encode_plus(
            row.source,
            None,
            add_special_tokens=True,
            max_length=MD_MAX_LEN,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        code_inputs = TOKENIZER.batch_encode_plus(
            [str(x) for x in row_fts["codes"]] or [""],
            add_special_tokens=True,
            max_length=23,
            padding="max_length",
            truncation=True,
        )

        ids = inputs["input_ids"]
        for x in code_inputs["input_ids"]:
            ids.extend(x[:-1])
        ids = ids[:TOTAL_MAX_LEN]
        if len(ids) != TOTAL_MAX_LEN:
            ids = ids + [
                TOKENIZER.pad_token_id,
            ] * (TOTAL_MAX_LEN - len(ids))

        mask = inputs["attention_mask"]
        for x in code_inputs["attention_mask"]:
            mask.extend(x[:-1])
        mask = mask[:TOTAL_MAX_LEN]
        if len(mask) != TOTAL_MAX_LEN:
            mask = mask + [
                TOKENIZER.pad_token_id,
            ] * (TOTAL_MAX_LEN - len(mask))

        input_ids[i] = ids
        attention_mask[i] = mask
        features[i] = (
            row_fts["total_md"] / (row_fts["total_md"] + row_fts["total_code"]) or 1
        )

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "features": features,
    }


In [63]:
df = convert_notebook(notebook_path=notebook_path)
df

,cell_type,source,id
cell_id,,,
a8a43ede,code,import pandas as pd\nimport numpy as np\nimpor...,test_notebook
3d474e2a,code,def read_notebook(path: str) -> pd.DataFrame:\...,test_notebook
d90ef310,markdown,# test test test afhjehfjeanfjenfnezfjkenf,test_notebook
dc46914f,markdown,,test_notebook
78bcb222,code,"def get_dataset(\n input_ids: np.array,\n ...",test_notebook
7780f4be,markdown,fehflezhfkezflkznelkfnzelkfnzeklfn,test_notebook
79392ebf,code,df = pd.read_csv('../raw_data/train_ancestors....,test_notebook
e2253470,markdown,jefkezjfkzejfkzejfkjzefkjzekfjzekfj,test_notebook
9213eaac,markdown,fuck off!,test_notebook


In [64]:
fts = get_features(df)
fts

Features:   0%|          | 0/1 [00:00<?, ?it/s]

{'test_notebook': {'total_code': 4,
  'total_md': 6,
  'codes': ['import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns',
   'def read_notebook(path: str) -> pd.DataFrame:\n    return (\n        pd.read_json(path, dtype={"cell_type": "category", "source": "str"})\n        .assign(id=os.path.basename(path).split(".")[0])\n        .rename_axis("cell_id")\n    )',
   'def get_dataset(\n    input_ids: np.array,\n    attention_mask: np.array,\n    feature: np.array,\n) -> tf.data.Dataset:\n    dataset = tf.data.Dataset.from_tensor_slices(\n        {"input_ids": input_ids, "attention_mask": attention_mask, "feature": feature}\n    )\n    dataset = dataset.batch(BATCH_SIZE)\n    return dataset.prefetch(tf.data.AUTOTUNE)',
   "df = pd.read_csv('../raw_data/train_ancestors.csv')\ndf.head(10)"]}}